In [13]:
from enum import Enum
import json
import yaml

from openai import BadRequestError, OpenAI, AsyncOpenAI
import pandas as pd
from pydantic import BaseModel

# Load Model

In [2]:
with open("prompts/try1.yaml", "r") as f:
    PROMPTS = yaml.safe_load(f)
SYSTEM_MESSAGE = PROMPTS["system"]
USER_TEMPLATE = PROMPTS["user"]

In [3]:
class RelationshipStatus(str, Enum):
    na = "na"
    low = "low"
    medium = "medium"
    high = "high"
    
class EmotionLabel(BaseModel):
    joy: RelationshipStatus
    trust: RelationshipStatus
    fear: RelationshipStatus
    surprise: RelationshipStatus
    sadness: RelationshipStatus
    disgust: RelationshipStatus
    anger: RelationshipStatus
    anticipation: RelationshipStatus
    
    # class Config:
    #     extra = Extra.forbid
    #     use_enum_values = True
        
class EntryResult(BaseModel):
    emotion: EmotionLabel
    reason: str

In [4]:
llm_model = "gpt-4.1-mini-2025-04-14"
df = pd.read_csv(f"data/comet/test_{llm_model}.tsv", sep="\t")
print(df.shape, df.columns)

(4452, 16) Index(['uid', 'original_idx', 'original_src', 'original_relation',
       'original_tgt', 'source', 'character', 'joy', 'trust', 'fear',
       'surprise', 'sadness', 'disgust', 'anger', 'anticipation', 'reason'],
      dtype='object')


In [ ]:
row = df.iloc[1]
user_message = USER_TEMPLATE.format(
    source=row['source'],
    character=row['character']
)
assistant_message = json.dumps(
    {
        "emotion": {
            "joy": row['joy'],
            "trust": row['trust'],
            "fear": row['fear'],
            "surprise": row['surprise'],
            "sadness": row['sadness'],
            "disgust": row['disgust'],
            "anger": row['anger'],
            "anticipation": row['anticipation']
        },
        "reason": row['reason']
    }
)

messages = [
    {"role": "system", "content": SYSTEM_MESSAGE},
    {"role": "user", "content": user_message},
    # {"role": "assistant", "content": assistant_message}
]

# Load Client

In [14]:
client = OpenAI(
    base_url="http://localhost:8010/v1",
    api_key="abc",
)

In [15]:
json_schema = EntryResult.model_json_schema()

completion = client.chat.completions.create(
    model="adapter",
    messages=messages,
    extra_body={"guided_json": json_schema},
)
print(completion.choices[0].message.content)

{
    "emotion": {
        "joy": "na",
        "trust": "na",
        "fear": "low",
        "surprise": "na",
        "sadness": "high",
        "disgust": "na",
        "anger": "low",
        "anticipation": "medium"
    },
    "reason": "Michael feels downtrodden due to the heavy workload and looming pressure to improve his project."
}


In [16]:
print("ANSWER:")
emotions = ["joy", "trust", "fear", "surprise", "sadness", "disgust", "anger", "anticipation"]
for emotion in emotions:
    print("{}: {}".format(emotion, row[emotion]))

ANSWER:
joy: na
trust: na
fear: low
surprise: na
sadness: high
disgust: na
anger: low
anticipation: medium
